In [1]:
import os
import sys
import pandas as pd


%pwd
os.chdir("../")

# Add the absolute path to src/ so Python can find automatch
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)
    
%pwd


'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [2]:
%pwd


'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [3]:
from autoMatch.utils.snowflake_utils import get_snowpark_session
session = get_snowpark_session()

[2025-10-01 17:14:55,367: INFO: connection: Snowflake Connector for Python Version: 3.7.0, Python Version: 3.10.11, Platform: Windows-10-10.0.26100-SP0]
[2025-10-01 17:14:55,369: INFO: connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.]
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/e2ba81b8-03fe-407c-96a1-f4bc0f512e7d/saml2?SAMLRequest=nZJBc9owEIX%2Fikc925IdA0YDZEiYtExJwwTIgZuQ10RFllxJxpBfX9mEmfSQHHrzyG%2F3e7tvR7enUgZHMFZoNUZxRFAAiutcqP0YbdYPYYYC65jKmdQKxugMFt1ORpaVsqLT2r2qZ%2FhTg3WBb6QsbX%2BMUW0U1cwKSxUrwVLH6Wr6uKBJRCizFozzOPReklvhWa%2FOVRTjpmmi5ibSZo8TQgg

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: str
    database: str
    schema: str
    input_table: str
    output_table: str
    columns: dict
    start_date: str
    end_date: str


In [5]:
from autoMatch.constants import *
from autoMatch.utils.common import read_yaml, create_directories
from autoMatch import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        schema = self.schema.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            database=config.database,
            schema=config.schema,
            input_table=config.input_table,
            output_table = config.output_table,
            columns = schema.columns,
            start_date = schema.date_range.start_date,
            end_date = schema.date_range.end_date
        )

        return data_ingestion_config

In [8]:
from snowflake.snowpark.functions import col

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def read_table(self, session):
        """
        Reads input table
        Function returns Snowflake dataframe
        """
        database = self.config.database
        schema = self.config.schema
        input_table = self.config.input_table
        columns = self.config.columns
        start_date = self.config.start_date
        end_date = self.config.end_date

        print(columns)
        df = session.table(f"{database}.{schema}.{input_table}")
        df = df.select([col(c) for c in columns])
        df = df.filter((col("date_added") >= start_date) & (col("date_added") <= end_date))
        logger.info(f"Table {input_table} successfully read. Number of rows: {df.count()}")

        return df


    def write_table(self, session, df):
        """
        Writes input table
        Function returns nothing
        """
        output_table = self.config.output_table

        #df.write.save_as_table(output_table, mode="overwrite")
        logger.info(f"Table {output_table} successfully written")

  


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    df = data_ingestion.read_table(session)
    data_ingestion.write_table(session, df)
except Exception as e:
    raise e

[2025-10-01 17:15:50,322: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-01 17:15:50,324: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-01 17:15:50,326: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-01 17:15:50,328: INFO: common: created directory at: artifacts]
[2025-10-01 17:15:50,331: INFO: common: created directory at: artifacts/data_ingestion]
['date_added', 'candidateid', 'description']
[2025-10-01 17:15:50,333: INFO: cursor: query: [SELECT  *  FROM IT_DISCOVERY.CONSUMER_INT_MODEL.CANDIDATE_CLEANED]]
[2025-10-01 17:15:51,999: INFO: cursor: query execution done]
[2025-10-01 17:15:52,001: INFO: cursor: Number of results in first chunk: 0]
[2025-10-01 17:15:52,009: INFO: cursor: query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( SELECT "DATE_ADDED", "CANDIDATEID",...]]
[2025-10-01 17:16:06,659: INFO: cursor: query execution done]
[2025-10-01 17:16:06,660: INFO: cursor: Number of results in first chunk: 1]
[2025-1